## part4_invest_metrics

除了无常损失之外，还可以引入一些其他的金融metrics来衡量头寸的收益。

### 1. ROI

$$ ROI = \frac {Net Profit} {Net Cost} = \frac { amount_{1} - amount_{0} } { amount_{0} }$$


处理成年华收益率：

$$ r_{annualized} = (1 + roi) ^ { \frac {365 } {days}} - 1 $$

### 2. volatility

计算头寸的波动率，需要头寸价值的时间序列数据。

当时间为i的时候，头寸的美元计算方式是加总池子内的两种token的美元价值，以及这段时间内产生的两种token的手续费的美元价值如下：

$$ V_i = p_0 \times (amount_0 + fee_0) + p_1 \times (amount_1 + fee_1) $$ 

其中：

i = 当前时间

amount_0 = 当前时间下头寸中token0的数量
amount_1 = 当前时间下头寸中token1的数量
fee_0 = 当前时间下token0 fee的数量
fee_1 = 当前时间下token1 fee的数量

### 3. Sharpe Ratio

Sharpe Ratio是投资组合超额收益与波动率之比，超额收益指的是投资组合的实际收益与无风险资产收益之差。Sharp ratio越高，表示投资组合的风险调整后的收益越高，表现越好。

\begin{equation*}
Sharpe\ Ratio = \frac{R_p - R_f}{\sigma_p}
\end{equation*}

其中：

$$ R_p = 投资组合收益率 $$

$$ R_f= = 无风险利率 $$

$$ \sigma_p = 投资组合标准差（即波动率 $$

** 这里有一个问题，加密货币市场中的无风险利率应如何定义呢？ **

在传统金融市场下，无风险利率一般用美国短期（一般是3个月）国债的YTM利率来表示，但加密市场没有国债，甚至没有一个真正稳定的稳定币，我们应该用什么来作为基准利率呢？
    思考：
    为什么美国短期国债可以被认定为是无风险的利率？
        无风险利率是衡量其他投资收益的基准利率，是没有风险的情况下，资本市场中的资产可以获得的收益。
        如果你以美元作为你的币本位，美国政府借入美元发行的债券就是无风险的，因为美国政府有发行货币的权利，始终可以印制更多美元来偿还债务。而如果是英国政府借入美元发行债券，严格来说这个利率就不能当作是无风险利率。
    
    以下是一些可能作为基准利率的选择：

    1.加密货币的借贷利率：许多加密货币交易所提供借贷服务，并提供借贷利率。这些借贷利率可以作为无风险利率的参考。

    2.加密货币期货/期权市场的利率：一些加密货币期货/期权市场提供无风险利率，这些利率可以作为无风险利率的参考。

    3.美元或其他法定货币的利率：虽然加密货币市场没有国债或稳定币，但可以使用美元或其他法定货币的利率作为无风险利率的参考。例如，可以使用美元短期国债的YTM利率作为无风险利率的参考。 
    
    如果以法币为中心，3.美元或其他法定货币的利率可能是最公平的一个选择。但是如果把defi作为一个独立的经济体来看，我们可以大胆地定义一些属于加密世界的原生无风险利率：

    1. 以eth为币本位，pos质押eth的收益作为无风险利率（类似于把eth网络看作一个国家）
        不过pos质押率中仍包含了其他风险的收益：
            运行pos节点故障被slash的风险，
        （lido等质押池子的收益率呢？）

    2. aave coumpoud等协议中，eth的借贷利率
        
        利率不高，大概是1.69%

        https://app.aave.com/reserve-overview/?underlyingAsset=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&marketName=proto_mainnet

        它也包含了其他风险的收益：
            违约风险
    
    3. flash loan借入eth的利率

        https://github.com/Jeiwan/flash-loans-comparison

        AAVE： 0.09% 

        UniswapV2: 0.3009027% fee


    2. 用对冲风险的方法得出无风险利率
        


### 4. impermanent loss

无常损失

$$ IL = \frac{Loss from LP}{hodl} = \frac { v_{1} - v_{hodl} }{ v_{hodl} } $$